<a href="https://colab.research.google.com/github/SaiSJoshi/SaiSJoshi/blob/main/train_vit_for_AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import sys
sys.path.append('/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/Code/vit')

In [3]:
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
! pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 15.6 MB/s 


In [5]:
! pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52.8 MB 111 kB/s 


In [6]:
import os
import sys 
import json
import glob
import random
import collections
import time
import re

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from torch.nn import functional as torch_functional
import torch.nn.functional as F

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [7]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(12)

In [8]:
from model import Model
from ad_net_dataset import ADNetDataset
from trainer import Trainer

In [9]:
train_dataset = ADNetDataset('/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/Code/vit/data/ad_train_ds', '/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/Code/vit/data/cn_train_ds')
test_dataset = ADNetDataset('/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/Code/vit/data/ad_test_ds', '/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/Code/vit/data/cn_test_ds')

In [10]:
# train_dataset[0]['X'].shape

In [11]:
train_loader = torch_data.DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=1,
    pin_memory = True,
)

test_loader = torch_data.DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=1,
    pin_memory = True,
)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Model(
    image_size = 128,
    patch_size = 32,
    num_classes = 1,
    dim = 1024,
    depth = 2,
    heads = 16,
    mlp_dim = 2048,
    channels = 1,
    dropout = 0.1,
    emb_dropout = 0.1
    
)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch_functional.binary_cross_entropy_with_logits


In [13]:
threshold = 0.5
trainer = Trainer(
    model, 
    device, 
    optimizer, 
    criterion,
    threshold_val=threshold
)

In [14]:
history = trainer.fit(
    10, 
    train_loader, 
    test_loader, 
    'out_models', 
    10,
)

EPOCH: 1
Valid Step 1/5, valid_loss: 0.5743
Valid Step 2/5, valid_loss: 1.1269
Valid Step 3/5, valid_loss: 0.8199
Valid Step 4/5, valid_loss: 0.8507
Valid Step 5/5, valid_loss: 0.9431
[Epoch Train: 1] loss: 0.992, time: 3.00 s            
[Epoch Valid: 1] loss: 0.943, auc: 0.500, time: 0.00 s, precision: 0.000, recall: 0.000, f1: 0.000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


auc improved from inf to 0.9431. Saved model to 'out_models-best.pth'
EPOCH: 2
Valid Step 1/5, valid_loss: 0.9357
Valid Step 2/5, valid_loss: 0.7565
Valid Step 3/5, valid_loss: 0.8162
Valid Step 4/5, valid_loss: 0.8462
Valid Step 5/5, valid_loss: 0.7565
[Epoch Train: 2] loss: 0.810, time: 1.00 s            
[Epoch Valid: 2] loss: 0.757, auc: 0.500, time: 0.00 s, precision: 0.000, recall: 0.000, f1: 0.000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


auc improved from 0.943 to 0.7565. Saved model to 'out_models-best.pth'
EPOCH: 3
Valid Step 1/5, valid_loss: 0.5747
Valid Step 2/5, valid_loss: 0.9465
Valid Step 3/5, valid_loss: 1.0747
Valid Step 4/5, valid_loss: 0.9492
Valid Step 5/5, valid_loss: 0.9498
[Epoch Train: 3] loss: 0.656, time: 1.00 s            
[Epoch Valid: 3] loss: 0.950, auc: 0.500, time: 0.00 s, precision: 0.500, recall: 1.000, f1: 0.667
EPOCH: 4
Valid Step 1/5, valid_loss: 0.7106
Valid Step 2/5, valid_loss: 0.7109
Valid Step 3/5, valid_loss: 0.7112
Valid Step 4/5, valid_loss: 0.7114
Valid Step 5/5, valid_loss: 0.7116
[Epoch Train: 4] loss: 0.834, time: 1.00 s            
[Epoch Valid: 4] loss: 0.712, auc: 0.500, time: 0.00 s, precision: 0.500, recall: 1.000, f1: 0.667
auc improved from 0.757 to 0.7116. Saved model to 'out_models-best.pth'
EPOCH: 5
Valid Step 1/5, valid_loss: 0.6979
Valid Step 2/5, valid_loss: 0.6978
Valid Step 3/5, valid_loss: 0.6819
Valid Step 4/5, valid_loss: 0.7094
Valid Step 5/5, valid_loss: 0.6

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


auc improved from 0.698 to 0.6963. Saved model to 'out_models-best.pth'
EPOCH: 7
Valid Step 1/5, valid_loss: 0.6556
Valid Step 2/5, valid_loss: 0.6764
Valid Step 3/5, valid_loss: 0.7108
Valid Step 4/5, valid_loss: 0.6968
Valid Step 5/5, valid_loss: 0.6970
[Epoch Train: 7] loss: 0.717, time: 1.00 s            
[Epoch Valid: 7] loss: 0.697, auc: 0.500, time: 0.00 s, precision: 0.500, recall: 1.000, f1: 0.667
EPOCH: 8
Valid Step 1/5, valid_loss: 0.7013
Valid Step 2/5, valid_loss: 0.7011
Valid Step 3/5, valid_loss: 0.7012
Valid Step 4/5, valid_loss: 0.7011
Valid Step 5/5, valid_loss: 0.7012
[Epoch Train: 8] loss: 0.721, time: 1.00 s            
[Epoch Valid: 8] loss: 0.701, auc: 0.500, time: 0.00 s, precision: 0.500, recall: 1.000, f1: 0.667
EPOCH: 9
Valid Step 1/5, valid_loss: 0.7169
Valid Step 2/5, valid_loss: 0.7173
Valid Step 3/5, valid_loss: 0.7175
Valid Step 4/5, valid_loss: 0.7056
Valid Step 5/5, valid_loss: 0.6939
[Epoch Train: 9] loss: 0.717, time: 1.00 s            
[Epoch Valid: